In [1]:
from glob import glob

import mlflow

# Q1

In [2]:
!mlflow --version

mlflow, version 2.13.0


# Q2

In [3]:
!python ./homework/preprocess_data.py \
    --raw_data_path "./data" \
    --dest_path "./output"

In [4]:
len(glob("./output/*"))

4

# Q3

In [5]:
!python homework/train.py --data_path "./output"

2024/05/24 22:54:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/mbonilla/miniconda3/envs/mlops_zoomcamp/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."


In [6]:
client = mlflow.client.MlflowClient()

run = client.search_runs(experiment_ids=["0"])[0]
run.data.params

{'bootstrap': 'True',
 'max_depth': '10',
 'max_samples': 'None',
 'min_weight_fraction_leaf': '0.0',
 'max_leaf_nodes': 'None',
 'min_samples_leaf': '1',
 'random_state': '0',
 'min_impurity_decrease': '0.0',
 'verbose': '0',
 'n_estimators': '100',
 'criterion': 'squared_error',
 'oob_score': 'False',
 'ccp_alpha': '0.0',
 'warm_start': 'False',
 'max_features': '1.0',
 'monotonic_cst': 'None',
 'n_jobs': 'None',
 'min_samples_split': '2'}

# Q4

```
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./artifacts
```

# Q5

In [7]:
!python homework/hpo.py

2024/05/24 22:54:19 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
100%|██████████| 15/15 [00:30<00:00,  2.02s/trial, best loss: 5.335419588556921]


In [8]:
tracking_uri = "http://127.0.0.1:5000"

client = mlflow.client.MlflowClient(tracking_uri=tracking_uri)
best_run = client.search_runs(
    experiment_ids=[1],
    order_by=["metrics.rmse ASC"]
)[0]

print(best_run.data.metrics)


{'rmse': 5.335419588556921}


# Q6

In [9]:
!python homework/register_model.py

2024/05/24 22:54:51 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2024/05/24 22:54:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/mbonilla/miniconda3/envs/mlops_zoomcamp/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'green-taxi-duration-model'.
2024/05/24 22:55:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: green-taxi-duration-model, version 1
Created version '1' of model 'green-taxi-duration-model'.


In [10]:
model = client.get_registered_model("green-taxi-duration-model")
client.get_run(model.latest_versions[0].run_id).data.metrics

{'training_mean_squared_error': 24.785660360279664,
 'training_mean_absolute_error': 3.23365652419934,
 'training_r2_score': 0.6956122194293166,
 'training_root_mean_squared_error': 4.978519896543516,
 'training_score': 0.6956122194293166,
 'val_rmse': 5.335419588556921,
 'test_rmse': 5.567408012462019}